In [47]:
from elasticsearch import Elasticsearch

In [48]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","qXCe9zdlLF*zJ_E8XAi7"),
    ca_certs="/Users/STEFANY TJUNG/Downloads/Information Retrieval/elasticsearch-8.13.4-windows-x86_64/elasticsearch-8.13.4/config/certs/http_ca.crt"
)
es.ping()

True

In [49]:
import pandas as pd

df = pd.read_csv("property.csv")
df.head()

,property_id,en_label,en_description
0,6,head of government,"head of the executive power of this town, city..."
1,10,video,"relevant video. For images, use the property P..."
2,14,traffic sign,"graphic symbol describing the item, used at th..."
3,15,route map,image of route map at Wikimedia Commons
4,16,highway system,system (or specific country specific road type...


In [50]:
df.isna().value_counts()

property_id  en_label  en_description
False        False     False             6905
                       True                80
Name: count, dtype: int64

In [51]:
df.fillna("Kosong", inplace=True)

In [52]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\STEFANY TJUNG\Downloads\Information Retrieval\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [53]:
df["en_descriptionvector"] = df["en_description"].apply(lambda x: model.encode(x))

In [54]:
df.head()

,property_id,en_label,en_description,en_descriptionvector
0,6,head of government,"head of the executive power of this town, city...","[0.0059736753, -0.019790918, 0.009836407, 0.05..."
1,10,video,"relevant video. For images, use the property P...","[0.0032795814, -0.013915496, 0.013628638, 0.02..."
2,14,traffic sign,"graphic symbol describing the item, used at th...","[-0.0051128073, -0.10314338, -0.010747547, 0.0..."
3,15,route map,image of route map at Wikimedia Commons,"[0.006565698, -0.04672547, -0.012105703, 0.073..."
4,16,highway system,system (or specific country specific road type...,"[0.004188626, 0.041369278, -0.015593521, 0.053..."


In [55]:
es.ping()

True

In [62]:
from indexMapping import indexMapping

es.indices.create(index="all_labels", mappings=indexMapping, request_timeout=1000000)

C:\Users\STEFANY TJUNG\AppData\Local\Temp\ipykernel_676\3806205177.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="all_labels", mappings=indexMapping, request_timeout=1000000)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_labels'})

In [63]:
record_list = df.to_dict("records")

In [68]:
for record in record_list:
    try:
        es.index(index="all_labels", document=record, id=record["property_id"])
    except Exception as e:
        print(e)

In [69]:
es.count(index="all_labels")

ObjectApiResponse({'count': 6985, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [74]:
input_keyword = "Currency"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "en_descriptionvector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 7668, 
}

res = es.knn_search(index="all_labels", knn=query , source=["en_label","en_description"])
res["hits"]["hits"]

C:\Users\STEFANY TJUNG\AppData\Local\Temp\ipykernel_676\1529121561.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_labels", knn=query , source=["en_label	","en_description"])


[{'_index': 'all_labels',
  '_id': '38',
  '_score': 0.6092143,
  '_source': {'en_label': 'currency',
   'en_description': 'currency used by item'}},
 {'_index': 'all_labels',
  '_id': '2121',
  '_score': 0.5635116,
  '_source': {'en_label': 'prize money',
   'en_description': 'amount in a specific currency'}}]